In [1]:
# 加载数据集
from PIL import Image
import json
from pathlib import Path
from torchvision import transforms

dataset_dir = '/data1/humw/Datasets/VGGFace2'

def load_data(data_dir, image_size=512, resample=2):
    import numpy as np
    def image_to_numpy(image):
        return np.array(image).astype(np.uint8)
    # more robust loading to avoid loaing non-image files
    images = [] 
    for i in list(Path(data_dir).iterdir()):
        if not i.suffix in [".jpg", ".png", ".jpeg"]:
            continue
        else:
            images.append(image_to_numpy(Image.open(i).convert("RGB")))
    images = [Image.fromarray(i).resize((image_size, image_size), resample) for i in images]
    images = np.stack(images)
    # from B x H x W x C to B x C x H x W
    images = torch.from_numpy(images).permute(0, 3, 1, 2).float()
    assert images.shape[-1] == images.shape[-2]
    return images

train_aug = [
        transforms.Resize(224, interpolation=transforms.InterpolationMode.BILINEAR),
        transforms.CenterCrop(224),
    ]
tensorize_and_normalize = [
    transforms.Normalize([0.5*255]*3,[0.5*255]*3),
]
all_trans = train_aug + tensorize_and_normalize
all_trans = transforms.Compose(all_trans)
    
# 加载模型
from transformers.models.clip.modeling_clip import CLIPVisionModelWithProjection
import torch
import os
import torch.nn.functional as F

device = "cuda:0"
torch_dtype = torch.bfloat16
pretrained_model_name_or_path = '/data1/humw/Pretrains/clip-vit-large-patch14'
model = CLIPVisionModelWithProjection.from_pretrained(pretrained_model_name_or_path).to(device, dtype=torch_dtype)
model.to(torch_dtype)
id_embeds_dict = {}
# 获取图像编码
person_id_list = sorted(os.listdir(dataset_dir))
for person_id in person_id_list:
    person_id_dir = os.path.join(dataset_dir, person_id, "set_B")
    clean_data = load_data(person_id_dir, 512, 2)
    original_data = clean_data.to(device).requires_grad_(False).to(dtype=torch_dtype)
    tran_original_data = all_trans(original_data)
    ori_embeds = model(tran_original_data, output_hidden_states=True).hidden_states[-2]
    id_embeds_dict[person_id] = ori_embeds
# 计算两两之间的编码余弦损失距离，距离越大越好
id_map_id = dict()
id_map_loss = dict()
for person_id_i in person_id_list:
    id_map_id[person_id_i] = -1
    id_map_loss[person_id_i] = -2
    for person_id_j in person_id_list:
        tmp = -F.cosine_similarity(id_embeds_dict[person_id_i], id_embeds_dict[person_id_j], -1).mean() # 越近越小，最小-1，越远越大，最大1
        if tmp > id_map_loss[person_id_i]:
            id_map_id[person_id_i] = person_id_j
            id_map_loss[person_id_i] = tmp

/data1/humw/anaconda3/envs/photomaker/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-20 20:16:41.803902: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-20 20:16:41.912298: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-20 20:16:41.942303: E tensorflow/stream_executor/cuda/cuda_blas.

In [2]:
# 保存json文件
dict = id_map_id
json.dump(dict,open('/data1/humw/Codes/FaceOff/max_clip_cosine_distance_map.json','w'), indent=4)